In [1]:
import os
import bye_cycle
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from beep.structure.cli import auto_load_processed

In [2]:
# Disabling tf warnings
tf.get_logger().setLevel('INFO')
# Disabling GPU
RUN_ON_GPU = True
if not RUN_ON_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=32*1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

8 Physical GPUs, 8 Logical GPUs


In [4]:
structure_path = '/home/ec2-user/SageMaker/data_cache/structure/'
#Get all structured files with the correct prefix
names = [os.path.join(structure_path, n) for n in os.listdir(structure_path) if "PreDiag" in n or "PredictionDiagnostics" in n]
#exclude stitched files
excuded_cells = [str(_) for _ in list(range(292,295)) + list(range(297,307))]
names = [n for n in names if (not "p2_structure" in n and ".gz" in n and not any(x in n for x in excuded_cells))]
N = len(names)
N

364

In [7]:
cell_one = auto_load_processed(names[0])
cell_two = auto_load_processed(names[-1])

In [9]:
cell_one.structured_summary['discharge_capacity']

0       4.750399
1       3.508421
2       3.498147
3       3.489705
4       3.481804
          ...   
1240    3.018786
1241    3.016508
1242    3.013734
1243    3.011360
1244    3.008897
Name: discharge_capacity, Length: 1245, dtype: float64

In [ ]:
n_points = 100
max_time_idx = bye_cycle.clean_cycle_data(cell_one, 10).shape[0]
max_cycle_idx = max(cell_one.structured_data['cycle_index'])
interp_clean_cell_one = [] #np.empty((max_cycle_idx, max_time_idx, 3))
empty_cycles = []
for i in tqdm(range(max_cycle_idx)):
    clean_dat = bye_cycle.clean_cycle_data(cell_one, i)
    # Looks like there are some missing cycle measurements so ...
    if clean_dat.shape[0] !=0:
        interp_clean_cell_one.append(bye_cycle.interpol_cycle(clean_dat, n_points=100))
    else:
        empty_cycles.append(i)
#     break
#     np.append(clean_cell_one, np.array(clean_cycle_data(cell_one, i))[:, np.newaxis, :], axis=0)
#     break
interp_clean_cell_one = np.array(interp_clean_cell_one)
print(f'Empty data found in cycles: {empty_cycles} in cell')

In [17]:
print(f'CACHED_DATA/cell_{int(i):03d}')

CACHED_DATA/cell_1314


In [18]:
print(i)
if os.path.exists(f'CACHED_DATA/cell_{int(i):03d}.pkl'):
    print(f'CACHED_DATA/cell_{int(i):03d}.pkl')

1314


,cycle_index,discharge_capacity,charge_capacity,discharge_energy,charge_energy,dc_internal_resistance,temperature_maximum,temperature_average,temperature_minimum,date_time_iso,energy_efficiency,charge_throughput,energy_throughput,charge_duration,time_temperature_integrated,paused,CV_time,CV_current
0,0,4.750399,3.870380,17.390939,15.072334,0.0,NaN,NaN,NaN,2020-12-19T20:19:16+00:00,1.153832,3.870380,15.072334,NaN,NaN,0,2062.030029,0.138247
1,6,3.508421,3.829267,12.907875,14.385632,0.0,NaN,NaN,NaN,2020-12-23T06:59:08+00:00,0.897276,7.699647,29.457966,NaN,NaN,0,1799.969971,0.224308
2,7,3.498147,3.513221,12.869881,13.314831,0.0,NaN,NaN,NaN,2020-12-23T15:10:20+00:00,0.966582,11.212869,42.772797,NaN,NaN,0,1799.979980,0.221103
3,8,3.489705,3.504323,12.839340,13.281834,0.0,NaN,NaN,NaN,2020-12-23T23:01:18+00:00,0.966684,14.717191,56.054630,NaN,NaN,0,1799.979980,0.222019
4,9,3.481804,3.496785,12.810638,13.254452,0.0,NaN,NaN,NaN,2020-12-24T06:51:14+00:00,0.966516,18.213976,69.309082,NaN,NaN,0,1799.989990,0.221332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,1310,3.018786,3.029007,11.107913,11.533662,0.0,NaN,NaN,NaN,2022-03-13T21:08:59+00:00,0.963086,3917.212891,14888.087891,NaN,NaN,0,1799.989990,0.189975
1241,1311,3.016508,3.025954,11.100614,11.522671,0.0,NaN,NaN,NaN,2022-03-14T04:01:08+00:00,0.963372,3920.238770,14899.610352,NaN,NaN,0,1799.989990,0.189059
1242,1312,3.013734,3.023249,11.091372,11.512824,0.0,NaN,NaN,NaN,2022-03-14T10:52:57+00:00,0.963393,3923.261963,14911.123047,NaN,NaN,0,1799.989990,0.192950
1243,1313,3.011360,3.020964,11.083298,11.504633,0.0,NaN,NaN,NaN,2022-03-14T17:44:25+00:00,0.963377,3926.282959,14922.627930,NaN,NaN,0,1799.989990,0.188830


In [52]:
# for i in range(5):
#     with open('CACHED_DATA/pre_processed_pandas/notes.txt', 'a') as f:
#         # Append 'hello' at the end of file
#         f.write(f'{i}. hello\n')

The printed cycles above are diagnostic cycles.

In [5]:
import pickle
structured_columns = ['voltage', 'current', 'cycle_index', 'discharge_capacity', 'charge_capacity']
summary_columns = ['cycle_index', 'discharge_capacity', 'charge_capacity', 'discharge_energy',
                   'charge_energy', 'charge_throughput', 'energy_throughput']
exclude_count = 0
for i,n in enumerate(names):
    structured_data = []
    summary_data = []
    notes = []
    # check to see if the cell was aleady excluded
    exclude_checker = False
    with open('CACHED_DATA/pre_processed_pandas/notes.txt', 'r') as f:
        if n[46:-8] in f.read():
            exclude_checker = True
            print(f'Cell {n[46:-8]} is already excluded. Please check notes.txt for more details.')
            
    if not exclude_checker:
        if not os.path.exists(f'CACHED_DATA/pre_processed_pandas/{n[46:-8]}.pkl'):
            cell = auto_load_processed(n)
            manufacturding_nominal_discharge_capacity = 4.84
            nominal_discharge_capacity_drop = 1- cell.structured_summary['discharge_capacity'][1]/manufacturding_nominal_discharge_capacity
            max_cycle_idx = max(cell.structured_data['cycle_index'])
            # excluding cells that that have more that 80% drop in discharge capacity within 1st cycle
            # excluding cells that that have less than 50 cycles
            cycle_idx_threshold = 50
            nominal_discharge_capacity_drop_threshold = 0.8
            if nominal_discharge_capacity_drop < nominal_discharge_capacity_drop_threshold and not max_cycle_idx < cycle_idx_threshold:
                print(f'Pre-processing data for cell {n[46:-8]}.')
                for c in tqdm(cell.structured_summary['cycle_index']):
                    clean_cell = bye_cycle.clean_cycle_data(cell, c, columns=structured_columns)
                    structured_data.append(clean_cell)
                summary_data.append(cell.structured_summary[summary_columns])
    #             features.append(bye_cycle.prep_features_per_cell(cell, n_points,
    #                                                              interpol_kind='quadratic',
    #                                                              columns=['voltage', 'current', 'cycle_index', 'discharge_capacity', 'charge_capacity']))

                with open(f'CACHED_DATA/pre_processed_pandas/{n[46:-8]}.pkl','wb') as f: 
                    pickle.dump([structured_data, summary_data], f) 
            else:
                if nominal_discharge_capacity_drop > nominal_discharge_capacity_drop_threshold:
                    add_txt = f'\n- Cell {n[46:-8]} was skipped as the nominal discharge capacity drop ({nominal_discharge_capacity_drop:.3f}) exceeds {nominal_discharge_capacity_drop_threshold}.\n'
                    print(add_txt)
                    with open('CACHED_DATA/pre_processed_pandas/notes.txt', 'r+') as f:
                        lines_list = f.readlines()
                        if add_txt not in lines_list: 
                            f.write(add_txt)
                if max_cycle_idx < cycle_idx_threshold:
                    add_txt = f'\n- Cell {n[46:-8]} was skipped as the cycle numbers ({max_cycle_idx}) is less than {cycle_idx_threshold}.\n'
                    print(add_txt)
                    with open('CACHED_DATA/pre_processed_pandas/notes.txt', 'r+') as f:
                        lines_list = f.readlines()
                        if add_txt not in lines_list: 
                            f.write(add_txt)

Cell PreDiag_000255_000214_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000218_00005F_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000206_000057_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000197_00020A_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000237_000222_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000204_00005C_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000252_00020E_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000264_0000D1_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000216_0001A1_structured is already excluded. Please check notes.txt for more details.
Cell PreDiag_000266_0000D5_structured is already excluded. Please check notes.txt for more details.


In [7]:
with open('CACHED_DATA/pre_processed_pandas/notes.txt', 'r') as f:
        lines_list = f.readlines()
lines_list = 

['Notes include information about the excluded cells in the pre-processing step.\n', '\n', '- Cell PreDiag_000255_000214_structured was skipped as the nominal discharge capacity drop (0.980) exceeds 0.8.\n', '\n', '- Cell PreDiag_000218_00005F_structured was skipped as the nominal discharge capacity drop (1.000) exceeds 0.8.\n', '\n', '- Cell PreDiag_000206_000057_structured was skipped as the nominal discharge capacity drop (1.000) exceeds 0.8.\n', '\n', '- Cell PreDiag_000197_00020A_structured was skipped as the nominal discharge capacity drop (0.851) exceeds 0.8.\n', '\n', '\n']


In [43]:
cell_two = auto_load_processed(names[1])
nominal_discharge_capacity_drop = 1- cell_two.structured_summary['discharge_capacity'][1]/cell_two.structured_summary['discharge_capacity'][0]
nominal_discharge_capacity_drop

0.9800935763621899

In [21]:
columns = ['voltage', 'current', 'cycle_index', 'discharge_capacity', 'charge_capacity']
max_cycle_idx = max(cell_one.structured_data['cycle_index'])
features = []
for j in tqdm(range(max_cycle_idx)):
    clean_cell = bye_cycle.clean_cycle_data(cell_one, j, columns=columns)
    features.append(clean_cell)

100%|██████████| 1315/1315 [01:13<00:00, 17.89it/s]


In [22]:
features

[      voltage   current  cycle_index  discharge_capacity  charge_capacity
 118  2.700801 -0.691582            0            4.750012          3.87038
 119  2.702502 -0.691505            0            4.749198          3.87038
 120  2.704204 -0.691269            0            4.748312          3.87038
 121  2.705906 -0.691463            0            4.747466          3.87038
 122  2.707608 -0.691463            0            4.746587          3.87038
 ..        ...       ...          ...                 ...              ...
 981  4.169369 -0.691937            0            0.000649          3.87038
 982  4.171071 -0.691647            0            0.000280          3.87038
 983  4.172773 -0.691279            0            0.000083          3.87038
 984  4.174474 -0.692459            0            0.000029          3.87038
 985  4.176176 -0.695295            0            0.000014          3.87038
 
 [868 rows x 5 columns],
 Empty DataFrame
 Columns: [voltage, current, cycle_index, discharge_capa

In [18]:
cell_one.structured_data[['voltage', 'current', 'cycle_index', 'discharge_capacity', 'charge_capacity']]

,voltage,current,cycle_index,discharge_capacity,charge_capacity
0,2.500000,NaN,0,NaN,NaN
1,2.501702,NaN,0,NaN,NaN
2,2.503403,NaN,0,NaN,NaN
3,2.505105,NaN,0,NaN,NaN
4,2.506807,NaN,0,NaN,NaN
...,...,...,...,...,...
2491995,NaN,NaN,1315,NaN,4.793597
2491996,NaN,NaN,1315,NaN,4.798414
2491997,NaN,NaN,1315,NaN,4.803232
2491998,NaN,NaN,1315,NaN,4.808050


In [12]:
cell_one.structured_summary['discharge_capacity']

0       4.750399
1       3.508421
2       3.498147
3       3.489705
4       3.481804
          ...   
1240    3.018786
1241    3.016508
1242    3.013734
1243    3.011360
1244    3.008897
Name: discharge_capacity, Length: 1245, dtype: float64

In [16]:
n = 1 - cell_one.structured_summary['discharge_capacity'][1]/cell_one.structured_summary['discharge_capacity'][0]
print(n)


0.26144715402645313


In [94]:
def prep_features_per_cell(cell, n_points=100, columns=['voltage', 'current', 'cycle_index'], interpol_kind='linear'):
    max_cycle_idx = max(cell.structured_data['cycle_index'])
    interp_clean_cell = []
    empty_cycles = []
    for i in tqdm(range(4346, max_cycle_idx)):
        clean_dat = bye_cycle.clean_cycle_data(cell, i, columns=columns)
        print(clean_dat.shape)
        # Looks like there are some missing cycle measurements so ...
        if clean_dat.shape[0] not in [0,1,2]:
            from scipy.interpolate import interp1d
            interpol_cell_cycle_data = []
            for i,c in enumerate(columns):
                x = np.arange(clean_dat[c].shape[0])
                print(x)
                f_out = interp1d(x, np.array([clean_dat[c]]), kind=interpol_kind)
                new_x = np.linspace(0,clean_dat[c].shape[0]-1, n_points)
                interpol_y = f_out(new_x)
                interpol_cell_cycle_data.append(interpol_y[0])
            interp_clean_cell.append(np.array(interpol_cell_cycle_data).T)
#             interp_clean_cell.append(bye_cycle.interpol_cycle(clean_dat, n_points=n_points, columns=columns, interpol_kind=interpol_kind))
        else:
            empty_cycles.append(i)
    interp_clean_cell = np.array(interp_clean_cell)
    return interp_clean_cell

In [95]:
prep_features_per_cell(cell, n_points, interpol_kind='quadratic',
                                                         columns=['voltage', 'current', 'cycle_index', 'discharge_capacity'])

  1%|          | 2/334 [00:00<00:26, 12.67it/s]

(2, 4)
(8, 4)
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
(10, 4)
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


  2%|▏         | 6/334 [00:00<00:26, 12.52it/s]

(9, 4)
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
(10, 4)
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
(9, 4)
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7 8]


  2%|▏         | 8/334 [00:00<00:26, 12.50it/s]

...

 99%|█████████▉| 330/334 [00:26<00:00, 12.36it/s]

(870, 4)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 2

100%|██████████| 334/334 [00:26<00:00, 12.49it/s]

(0, 4)
(0, 4)
(0, 4)


array([[[ 3.501e+00, -9.680e+00,  4.347e+03,  6.154e-04],
        [ 3.501e+00, -9.680e+00,  4.347e+03,  6.064e-04],
        [ 3.501e+00, -9.680e+00,  4.347e+03,  5.977e-04],
        ...,
        [ 3.512e+00, -9.681e+00,  4.347e+03,  1.410e-04],
        [ 3.512e+00, -9.681e+00,  4.347e+03,  1.365e-04],
        [ 3.513e+00, -9.681e+00,  4.347e+03,  1.321e-04]],

       [[ 3.501e+00, -9.680e+00,  4.348e+03,  6.705e-04],
        [ 3.501e+00, -9.680e+00,  4.348e+03,  6.553e-04],
        [ 3.501e+00, -9.680e+00,  4.348e+03,  6.407e-04],
        ...,
        [ 3.516e+00, -9.684e+00,  4.348e+03,  1.263e-04],
        [ 3.516e+00, -9.684e+00,  4.348e+03,  1.217e-04],
        [ 3.516e+00, -9.684e+00,  4.348e+03,  1.171e-04]],

       [[ 3.501e+00, -9.680e+00,  4.349e+03,  7.033e-04],
        [ 3.501e+00, -9.680e+00,  4.349e+03,  6.903e-04],
        [ 3.501e+00, -9.680e+00,  4.349e+03,  6.776e-04],
        ...,
        [ 3.514e+00, -9.683e+00,  4.349e+03,  1.424e-04],
        [ 3.514e+00, -9.683e+

In [83]:
n_points=100
columns=['voltage', 'current', 'cycle_index']
interpol_kind='linear'

In [84]:

from scipy.interpolate import interp1d
interpol_cell_cycle_data = []
for i,c in enumerate(columns):
    x = np.arange(clean_dat[c].shape[0])
    f_out = interp1d(x, np.array([clean_dat[c]]), kind=interpol_kind)
    new_x = np.linspace(0,clean_dat[c].shape[0]-1, n_points)
    interpol_y = f_out(new_x)
    interpol_cell_cycle_data.append(interpol_y[0])
np.array(interpol_cell_cycle_data).T

array([[ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3.501e+00, -9.682e+00,  4.346e+03],
       [ 3

In [33]:
def prep_features_per_cell(cell, n_points=100, columns=['voltage', 'current', 'cycle_index'], interpol_kind='linear'):
    max_cycle_idx = max(cell.structured_data['cycle_index'])
    interp_clean_cell = []
    empty_cycles = []
    for i in tqdm(range(max_cycle_idx)):
        clean_dat = bye_cycle.clean_cycle_data(cell, i, columns=columns)
        # Looks like there are some missing cycle measurements so ...
        if clean_dat.shape[0] not in [0,1]:
            interp_clean_cell.append(bye_cycle.interpol_cycle(clean_dat, n_points=n_points, columns=columns, interpol_kind=interpol_kind))
        else:
            empty_cycles.append(i)
    interp_clean_cell = np.array(interp_clean_cell)
    return interp_clean_cell

In [68]:
max_cycle_idx = max(cell.structured_data['cycle_index'])
clean_dat = bye_cycle.clean_cycle_data(cell, 4346)
a = bye_cycle.interpol_cycle(clean_dat, n_points=n_points, columns=columns, interpol_kind='linear')
if clean_dat.shape[0] not in [0,1]:
            interp_clean_cell.append(bye_cycle.interpol_cycle(clean_dat, n_points=n_points, columns=columns, interpol_kind='linear'))

In [66]:
interp_clean_cell

[array([[ 3.501e+00, -9.682e+00,  4.346e+03,  1.631e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.628e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.625e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.622e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.619e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.616e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.613e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.610e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.607e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.604e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.601e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.598e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.595e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.592e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.589e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.586e-04],
        [ 3.501e+00, -9.682e+00,  4.346e+03,  1.583e-04],
        [ 3.50

In [49]:
columns=['voltage', 'current', 'cycle_index', 'discharge_capacity']
selected_cell_data = cell.structured_data[columns].dropna()
clean_cell_data_cycle_number = selected_cell_data.loc[(selected_cell_data['cycle_index'] == 4345) ]
#                                               & (selected_cell_data['discharge_capacity']!=0)]

In [50]:
clean_cell_data_cycle_number

,voltage,current,cycle_index,discharge_capacity


In [70]:
interp_clean_cell = []
interp_clean_cell.append(bye_cycle.interpol_cycle(clean_dat, n_points=n_points))

In [14]:
np.array(features).shape

(1, 1245, 100, 4)

In [16]:
np.array(features)[0,0,:,0]

array([2.701, 2.716, 2.731, 2.746, 2.76 , 2.775, 2.79 , 2.805, 2.82 ,
       2.835, 2.85 , 2.865, 2.88 , 2.895, 2.909, 2.924, 2.939, 2.954,
       2.969, 2.984, 2.999, 3.014, 3.029, 3.044, 3.058, 3.073, 3.088,
       3.103, 3.118, 3.133, 3.148, 3.163, 3.178, 3.193, 3.207, 3.222,
       3.237, 3.252, 3.267, 3.282, 3.297, 3.312, 3.327, 3.342, 3.357,
       3.371, 3.386, 3.401, 3.416, 3.431, 3.446, 3.461, 3.476, 3.491,
       3.506, 3.52 , 3.535, 3.55 , 3.565, 3.58 , 3.595, 3.61 , 3.625,
       3.64 , 3.655, 3.669, 3.684, 3.699, 3.714, 3.729, 3.744, 3.759,
       3.774, 3.789, 3.804, 3.819, 3.833, 3.848, 3.863, 3.878, 3.893,
       3.908, 3.923, 3.938, 3.953, 3.968, 3.982, 3.997, 4.012, 4.027,
       4.042, 4.057, 4.072, 4.087, 4.102, 4.117, 4.131, 4.146, 4.161,
       4.176])

In [20]:
# This cell takes forever to run if you don't use CACHED Data!
import pickle
USE_CACHED_DATA = True
features_file_name = 'features.pkl'
if not USE_CACHED_DATA:
    features = []
    for i,n in enumerate(names):
        cell = auto_load_processed(n)
        print(f'Featurizing data for cell {i}.')
        features.append(prep_features_per_cell(cell, n_points))
    with open(features_file_name, 'wb') as f: 
        pickle.dump(features, f)
else:
    with open(features_file_name, 'rb') as f:
        features = np.array(pickle.load(f))
        print('CACHED data is loaded for features.')

512
1315


In [21]:
n_points = 100
max_time_idx = bye_cycle.clean_cycle_data(cell_two, 10).shape[0]
max_cycle_idx = max(cell_two.structured_data['cycle_index'])
interp_clean_cell_two = [] #np.empty((max_cycle_idx, max_time_idx, 3))
empty_cycles = []
for i in tqdm(range(max_cycle_idx)):
    clean_dat = bye_cycle.clean_cycle_data(cell_two, i)
    # Looks like there are some missing cycle measurements so ...
    if clean_dat.shape[0] !=0:
        interp_clean_cell_two.append(bye_cycle.interpol_cycle(clean_dat, n_points=100))
    else:
        empty_cycles.append(i)
#     break
#     np.append(clean_cell_one, np.array(clean_cycle_data(cell_one, i))[:, np.newaxis, :], axis=0)
#     break
interp_clean_cell_two = np.array(interp_clean_cell_two)
print(f'Empty data found in cycles: {empty_cycles} in cell')

100%|██████████| 363/363 [00:06<00:00, 55.50it/s]

Empty data found in cycles: [1, 2, 3, 4, 5, 36, 37, 38, 39, 40, 141, 142, 143, 144, 145, 246, 247, 248, 249, 250, 351, 352, 353, 354, 355, 358, 359, 360, 361, 362] in cell


In [22]:
print(max_time_idx)
print(max_cycle_idx)

966
363
